# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from fastcore.basics import patch_to
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ) -> List[reader.Entry]:
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)
                if filter_string in i.link:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

    def _mark_important(self, feed_url: str, entry_id: str):
        article = r._reader.get_entry((feed_url, entry_id))
        self._reader.mark_entry_as_important(article)

    def _mark_unimportant(self, feed_url: str, entry_id: str):
        article = r._reader.get_entry((feed_url, entry_id))
        self._reader.mark_entry_as_unimportant(article)

In [ ]:
# | export


@patch_to(PyNewsReader)
def blacklist_add(self, blacklist_string: str):
    """Add entry to blacklist"""
    if blacklist_string not in self._title_blacklist:
        self._title_blacklist.append(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_remove(self, blacklist_string: str):
    """Remove entry from blacklist"""
    if blacklist_string in self._title_blacklist:
        self._title_blacklist.remove(blacklist_string)
        with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
            json.dump(self._title_blacklist, myfile)


@patch_to(PyNewsReader)
def blacklist_show(self):
    """Show blacklist"""
    print(self._title_blacklist)

In [ ]:
#| export


@patch_to(PyNewsReader)
def whitelist_add(self, whitelist_string: str):
    """Add entry to whitelist"""
    if whitelist_string not in self._title_whitelist:
        self._title_whitelist.append(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_important(entry)


@patch_to(PyNewsReader)
def whitelist_remove(self, whitelist_string: str):
    """Remove whitelist entry"""
    if whitelist_string in self._title_whitelist:
        self._title_whitelist.remove(whitelist_string)
        with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
            json.dump(self._title_whitelist, myfile)
        for entry in self._get_entries(limit=None):
            if whitelist_string in entry.title:
                self._reader.mark_entry_as_unimportant(entry)


@patch_to(PyNewsReader)
def whitelist_show(self):
    """Show whitelist"""
    print(self._title_whitelist)

In [ ]:
# | export


@patch_to(PyNewsReader)
def update(self):
    """Update feeds and search"""
    self._reader.update_feeds()
    self._reader.update_search()
    if len(self._title_blacklist) > 0:
        self._mark_matching_entries_as_read(self._title_blacklist)
    if len(self._title_whitelist) > 0:
        self._mark_matching_entries_as_important(self._title_whitelist)

In [ ]:
# | export


@patch_to(PyNewsReader)
def add_feed(self, feed: Union[Feed, str]):
    """Add feed to pynewsreader

    Args:
        feed (Feed): pynewsreader Feed to add
    """
    if isinstance(feed, Feed):
        self._feed_names[feed.url] = feed.name
        self._reader.add_feed(feed.url.rstrip("/"), exist_ok=True)
    elif isinstance(feed, str):
        self._reader.add_feed(feed.rstrip("/"), exist_ok=True)
    else:
        raise Exception("Must be str or Feed type to add")

    # Save names to file
    with open(self._dbfolder / "feed_names.json", "wt") as myfile:
        json.dump(self._feed_names, myfile)


@patch_to(PyNewsReader)
def remove_feed(self, feed: Union[Feed, str]):
    """Remove feed from pynewsreader instance

    Args:
        feed (Union[Feed, str]): Feed to remove
    """

    if isinstance(feed, Feed):
        self._reader.delete_feed(feed.url)
    elif isinstance(feed, str):
        self._reader.delete_feed(feed)
    else:
        raise Exception(TypeError)


@patch_to(PyNewsReader)
def feeds(self):
    """List pynewsreader feeds

    Returns:
        List[str]: List of names of current pynewsreader feeds
    """
    return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

In [ ]:
# | export


@patch_to(PyNewsReader)
def show(
    self,
    limit: int = 6,
    read: bool = False,
    important: bool = None,
    mark_as_read: bool = True,
):
    """Pretty print entries

    Args:
        limit (int, optional): Number of entries to show. Defaults to 5.
        read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
        mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
    """
    self._print_entries(
        self._get_entries(read=read, important=important, limit=limit * 2),
        limit=limit,
        mark_as_read=mark_as_read,
    )

In [ ]:
# | export


@patch_to(PyNewsReader)
def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
    """Search entries and pretty print results

    Args:a
        query (str): Search query
        mark_as_read (bool, optional): Mark results as read? Defaults to True.
    """
    self._print_entries(
        [self._reader.get_entry(i) for i in self._reader.search_entries(query)],
        mark_as_read=mark_as_read,
        limit=limit,
    )

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
    "https://www.theguardian.com/science/rss",
    "https://www.theguardian.com/world/canada/rss",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

## List Feeds

In [ ]:
r.feeds()

['Canada | The Guardian',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Facebook AI Blog',
 'Google AI Blog',
 'Huggingface Blog',
 'Journal of Statistical Software',
 'Macleans.ca',
 'Macleans',
 'OpenAI Blog',
 'Posit AI Blog',
 'Proceedings of Machine Learning Research',
 'Richochet Media',
 'Science | The Guardian',
 'Technology | The Guardian',
 'The Beaverton',
 'The Berkeley Artificial Intelligence Research Blog',
 'The Guardian',
 'The Narwhal',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: New Zealand announces its biggest emissions reduction project in history                                 │
│ Date: 2023-05-22                                                                                                │
│                                                                                                                 │
│ Move to power Glenbrook steel plant with electricity from renewables rather than coal will reduce emissions by  │
│ 1% – or the equivalent of taking 300,000 cars off the roadNew Zealand has announced its largest emissions       │
│ reduction project in history, transitioning from coal to renewable electricity at the country’s major steel     │
│ plant in a move that the government says is equivalent to taking 300,000 cars off the road.The government will  │
│ spend $140m on halving the coal used at Glenbrook steel plant to recycle scrap steel, replacing that generating │
│ power with an electric-powered furnace. The plant will contribute $160m to the project’s cost. Continue         │
│ reading...                                                                                                      │
│                                                                                                                 │
╰─ ]8;id=964411;https://www.theguardian.com/world/2023/may/22/new-zealand-announces-its-biggest-emissions-reduction-project-in-history\https://www.theguardian.com/world/2023/may/22/new-zealand-announces-its-biggest-emissions-reduction-project-in]8;;\─╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: US and Papua New Guinea set to sign security agreement amid Pacific militarisation concerns              │
│ Date: 2023-05-22                                                                                                │
│                                                                                                                 │
│ Antony Blinken has arrived on the Pacific island for signing as PM James Marape defends need to strengthen      │
│ bordersThe United States is scheduled to sign a new security pact with Papua New Guinea on Monday amid concerns │
│ in PNG about increasing militarisation and as the US continues to compete with China for influence in the       │
│ Pacific.The state department said the new agreement would provide a framework to help improve security          │
│ cooperation, enhance the capacity of Papua New Guinea’s defence force and increase regional stability. Continue │
│ reading...                                                                                                      │
│                                                                                                                 │
╰─ ]8;id=482449;https://www.theguardian.com/world/2023/may/22/us-png-defence-security-agreement-papua-new-guinea-china-india-modi-pacific-militarisation-concerns\https://www.theguardian.com/world/2023/may/22/us-png-defence-security-agreement-papua-new-guinea-china-india-m]8;;\─╯

╭──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╮
│ Title: The National | Zelenskyy at G7, Bakhmut victory dispute, Smoky Alberta                                   │
│ Date: 2023-05-22                                                                                                │
│                                                                                                                 │
│ May 21, 2023 | Ukraine’s president secures more Western aid at the G7 summit in Japan. Ukrainian forces dispute │
│ Russian victory claims in Bakhmut. Plus Alberta firefighters look for rain as wildfire smoke spreads.           │
│                                                                                                                 │
╰─ ]8;id=176661;https://www.cbc.ca/news/thenational/the-national-zelenskyy-at-g7-bakhmut-victory-dispute-smoky-alberta-1.6589288?cmp=rss\https://www.cbc.ca/news/thenational/the-national-zelenskyy-at-g7-bakhmut-victory-dispute-smoky-alberta-1.65892]8;;\─╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: How Smith’s Words and Actions Made ‘Trust’ a Ballot Issue                                                │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ Fit to govern? Here’s what Alberta’s UCP leader has done to spark the question.                                 │
│                                                                                                                 │
╰────────────── ]8;id=657768;https://thetyee.ca/News/2023/05/19/How-Smith-Words-Actions-Made-Trust-Ballot-Issue/\https://thetyee.ca/News/2023/05/19/How-Smith-Words-Actions-Made-Trust-Ballot-Issue/]8;;\ ──────────────╯

╭─────────────────────────────────────────────────── The Tyee ────────────────────────────────────────────────────╮
│ Title: Danielle Smith’s Day Went Bad Long Before the Debate                                                     │
│ Date: Unknown                                                                                                   │
│                                                                                                                 │
│ A damning ethics report and dumped candidate upstaged a leaders’ debate draw.                                   │
│                                                                                                                 │
╰─────────────────────── ]8;id=442287;https://thetyee.ca/Opinion/2023/05/19/Danielle-Smith-Day-Debate/\https://thetyee.ca/Opinion/2023/05/19/Danielle-Smith-Day-Debate/]8;;\ ────────────────────────╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Inflation rate unexpectedly increased in April, jumping up to 4.4%                                       │
│ Date: 2023-05-16                                                                                                │
│                                                                                                                 │
│ Canada's inflation rate reversed its cooling trend last month and moved higher, to a 4.4 per cent annual rate.  │
│                                                                                                                 │
╰────────────────────── ]8;id=233999;https://www.cbc.ca/news/business/inflation-april-1.6844703?cmp=rss\https://www.cbc.ca/news/business/inflation-april-1.6844703?cmp=rss]8;;\ ───────────────────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ ❗Title: Bank of Canada raises interest rate again — but hits 'pause' on any more for now                       │
│ Date: 2023-01-25                                                                                                │
│                                                                                                                 │
│ The Bank of Canada has raised its benchmark interest rate again, to 4.5 per cent, but hinted that it may now be │
│ done with its aggressive campaign of rate hikes.                                                                │
│                                                                                                                 │
╰──────────── ]8;id=325398;https://www.cbc.ca/news/business/bank-of-canada-january-rate-decision-1.6725283?cmp=rss\https://www.cbc.ca/news/business/bank-of-canada-january-rate-decision-1.6725283?cmp=rss]8;;\ ────────────╯

╭─────────────────────────────────────────── Toronto Star | Editorials ───────────────────────────────────────────╮
│ ❗Title: With inflation easing will interest rates hikes pause?                                                 │
│ Date: 2023-01-20                                                                                                │
│                                                                                                                 │
│ With some welcome downward movement on inflation, let’s hope then that we’re seeing an end to the Bank of       │
│ Canada’s string of rate hikes.                                                                                  │
│                                                                                                                 │
╰─ ]8;id=979120;https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.html\https://www.thestar.com/opinion/editorials/2023/01/20/with-inflation-easing-will-interest-rates-hikes-pause.ht]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: High office vacancy rates spell continuing trouble for shops dependent on worker foot traffic            │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ Businesses that depend on the flow of office workers are struggling as vacancy rates in office towers across    │
│ Canada continue to rise. Vancouver's office tower vacancy rate has tripled since 2019, while Montreal's has     │
│ doubled.                                                                                                        │
│                                                                                                                 │
╰────────── ]8;id=195533;https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss\https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss]8;;\ ──────────╯

╭────────────────────────────────────────── Toronto Star | Top Stories ───────────────────────────────────────────╮
│ ❗Title: ‘Vicious feedback loop’: The Bank of Canada hiked rates to push inflation down — but it’s also driving │
│ inflation up                                                                                                    │
│ Date: 2023-05-18                                                                                                │
│                                                                                                                 │
│ As more Canadians renewed their mortgages at much higher interest rates, costs have soared and contributed to   │
│ sticky inflation.                                                                                               │
│                                                                                                                 │
╰─ ]8;id=980269;https://www.thestar.com/business/2023/05/18/mortgage-costs-jump-28-per-cent-in-april-driving-an-inflation-feedback-loop.html\https://www.thestar.com/business/2023/05/18/mortgage-costs-jump-28-per-cent-in-april-driving-an-inflation-feed]8;;\─╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: More U.S. rate hikes in store as Federal Reserve warns inflation could 'spring back'                     │
│ Date: 2023-02-02                                                                                                │
│                                                                                                                 │
│ It's premature to declare victory over inflation, says the U.S. central bank's Jerome Powell, in a move that    │
│ could force North American interest rates even higher yet.                                                      │
│                                                                                                                 │
╰─────────────── ]8;id=757571;https://www.cbc.ca/news/business/fed-outlook-column-don-pittis-1.6733151?cmp=rss\https://www.cbc.ca/news/business/fed-outlook-column-don-pittis-1.6733151?cmp=rss]8;;\ ────────────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: U.S. added 517,000 jobs last month, pushing unemployment rate to lowest since 1969                       │
│ Date: 2023-02-03                                                                                                │
│                                                                                                                 │
│ The U.S. economy added more than half a million jobs in January, an astonishing number that pushed the jobless  │
│ rate down to its lowest level in half a century.                                                                │
│                                                                                                                 │
╰────────────────────── ]8;id=686916;https://www.cbc.ca/news/business/us-jobs-january-1.6735707?cmp=rss\https://www.cbc.ca/news/business/us-jobs-january-1.6735707?cmp=rss]8;;\ ───────────────────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Official inflation rate cools to 6.8% — but just about everything except gasoline keeps getting more     │
│ expensive                                                                                                       │
│ Date: 2022-12-21                                                                                                │
│                                                                                                                 │
│ Canada's inflation rate cooled to 6.8 per cent last month as prices for gasoline and furniture went down, but   │
│ the cost of food and rent went up.                                                                              │
│                                                                                                                 │
╰────────────────────── ]8;id=402474;https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss\https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss]8;;\ ──────────────────────╯

╭─────────────────────────────────────────────── CBC | Ottawa News ───────────────────────────────────────────────╮
│ Title: Death rate among on-duty police officers fallen since 1960s, new data shows, with car crashes top cause  │
│ Date: 2023-05-20                                                                                                │
│                                                                                                                 │
│ New data from University of Ottawa researchers shows the death rate for officers on the job has fallen in past  │
│ decades. It also shows how they died. Historically, car crashes have killed more officers than guns — though    │
│ that hasn't been the case recently.                                                                             │
│                                                                                                                 │
╰──────── ]8;id=456744;https://www.cbc.ca/news/canada/hamilton/police-officer-deaths-trend-research-1.6834737?cmp=rss\https://www.cbc.ca/news/canada/hamilton/police-officer-deaths-trend-research-1.6834737?cmp=rss]8;;\ ─────────╯

╭───────────────────────────────────────────────── The Guardian ──────────────────────────────────────────────────╮
│ Title: ‘This is just devilish … I’m going in again’: the best hot sauces tasted and rated                       │
│ Date: 2023-05-21                                                                                                │
│                                                                                                                 │
│ Who best to test 15 hot sauces – from mild and fruity to fierce and fiery – in a single sitting? Food TV’s Big  │
│ Zuu, that’s who£6.45, patandpinkys.com Continue reading...                                                      │
│                                                                                                                 │
╰─ ]8;id=464235;https://www.theguardian.com/food/2023/may/21/this-is-just-devilish-the-best-hot-sauces-tasted-and-rated-big-zuu\https://www.theguardian.com/food/2023/may/21/this-is-just-devilish-the-best-hot-sauces-tasted-and-rated-big-zu]8;;\─╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._mark_important(test[0].feed_url, test[0].id)

In [ ]:
r._mark_unimportant(test[0].feed_url, test[0].id)

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in [
    "Musk",
    "Apple",
    "Bezos",
    "Google",
    "Samsung",
    "iPhone",
    "iPad",
    "guardian.com/sport/",
    "guardian.com/football/",
    "thestar.com/sports",
]:
    r.blacklist_add(i)

In [ ]:
r.blacklist_show()

[
    'Musk',
    'Apple',
    'Bezos',
    'Google',
    'Samsung',
    'iPhone',
    'iPad',
    'Premier League',
    'NHL',
    'NFL',
    'NBA',
    'guardian.com/sport/',
    'guardian.com/football/',
    'thestar.com/sports'
]

In [ ]:
# r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r.whitelist_add(i)

In [ ]:
r.whitelist_show()

['interest rate', 'Bank of Canada', 'housing market']

In [ ]:
# r.update()

## Low level access to data

In [ ]:
entries = r._get_entries(limit=1)
entry = [i for i in entries][0]
entry

Entry(id='https://www.theguardian.com/world/2023/may/22/new-zealand-announces-its-biggest-emissions-reduction-project-in-history', updated=datetime.datetime(2023, 5, 22, 1, 7, 15, tzinfo=datetime.timezone.utc), title='New Zealand announces its biggest emissions reduction project in history', link='https://www.theguardian.com/world/2023/may/22/new-zealand-announces-its-biggest-emissions-reduction-project-in-history', author='Tess McClure in Auckland', published=datetime.datetime(2023, 5, 22, 1, 7, 15, tzinfo=datetime.timezone.utc), summary='<p>Move to power Glenbrook steel plant with electricity from renewables rather than coal will reduce emissions by 1% – or the equivalent of taking 300,000 cars off the road</p><p>New Zealand has announced its largest emissions reduction project in history, transitioning from coal to renewable electricity at the country’s major steel plant in a move that the government says is equivalent to taking 300,000 cars off the road.</p><p>The government will s

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | export


if __name__ == "__main__":
    main()

OSError: source code not available

In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()